<a href="https://colab.research.google.com/github/Charmaineem/ML/blob/main/Handwritten_digits_using_Logistic_Regression_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Identifying handwritten digits using Logistic Regression in PyTorch

How to classify handwritten digits from the MNIST dataset

Has a training set of 60,000 examples, and a test set of 10,000 examples

In [28]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [29]:
transform = transforms.Compose(
    transforms.ToTensor()
)

train_dataset = datasets.MNIST(
    root='data',
    download=True,
    train=True,
    transform=transform
)

test_dataset = datasets.MNIST(
    root='data',
    download=True,
    train=False,
    transform=transform
)

In [30]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [33]:
input_size = 784
num_classes = 10
num_epochs = 5
learning_rate = 0.001